In [ ]:
import waffles
import numpy as np
import json
import shutil 
from tqdm import tqdm

from waffles.input_output.hdf5_structured import load_structured_waveformset
from waffles.data_classes.Waveform import Waveform
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.data_classes.BasicWfAna import BasicWfAna
from waffles.data_classes.IPDict import IPDict
from waffles.data_classes.UniqueChannel import UniqueChannel
from waffles.utils.baseline.baseline import SBaseline
from waffles.np02_utils.AutoMap import generate_ChannelMap, dict_uniqch_to_module, dict_module_to_uniqch
from waffles.np02_utils.PlotUtils import np02_gen_grids, plot_grid


In [ ]:
run = 37218
nwaveforms = 80000
dettype = "membrane"
# dettype = "cathode"
trigger = "self_trigger"
# trigger = "full_streaming"
# hostname = 'np04-srv-004'

## Only change if necessary
output_dir = f"/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-VD/commissioning/"
ch = {}
save_single_file = False
max_files  = 4
det = "VD_Cathode_PDS" if dettype == "cathode" else "VD_Membrane_PDS"
endpoint = 106 if dettype == "cathode" else 107

# Way to low... keep scrollng
dletter = dettype.upper()[0] # C or M...
group1 = [ f"{dletter}{detnum}({chnum})" for detnum in range(1, 3) for chnum in range(1,3) ]
group2 = [ f"{dletter}{detnum}({chnum})" for detnum in range(3, 5) for chnum in range(1,3) ]
group3 = [ f"{dletter}{detnum}({chnum})" for detnum in range(5, 7) for chnum in range(1,3) ]
group4 = [ f"{dletter}{detnum}({chnum})" for detnum in range(7, 9) for chnum in range(1,3) ]

In [ ]:
wafflespath = waffles.__path__[0]
wafflespath = wafflespath.replace("src/waffles", "scripts")
configfile = shutil.copy(f"{wafflespath}/config.json", f"{wafflespath}/snrcheck_config.json")
with open(configfile, "r") as f:
    configdata = json.load(f)

configdata['runs'] = [run]
configdata['output_dir'] = output_dir
configdata['save_single_file'] = save_single_file
configdata['det'] = det
configdata['trigger'] = trigger
configdata['ch'] = ch
configdata['max_files'] = max_files

with open(configfile, "w") as f:
    json.dump(configdata, f, indent=4)
    print(f"Config file saved to {configfile}")

from pathlib import Path
plotntco = Path(output_dir) / f"plots/run{run:06d}_{dettype}/nTCO.html"
plottco = Path(output_dir) / f"plots/run{run:06d}_{dettype}/TCO.html"

doplot = "true"
# comment in case you want to force plotting
if plotntco.exists() and plottco.exists():
    print(f"Plots already exist:\n{plotntco}\n{plottco}")
    doplot = "false"

In [ ]:
%%bash -s "$wafflespath" "$run" "$configfile" "$doplot" "$hostname" "$trigger"

wafflespath=$1
run=$2
configjson=$3
doplot=$4
hostname=$5
trigger=$6

user=$( whoami )
script08="${wafflespath}/08_np02vd_run_downloader_processor.py"
configfile="${wafflespath}/${configjson}"
if [ "$trigger" != "self_trigger" ]; then
    echo "Please, run the following command in a terminal... IO not supported in notebook"
    echo ""
    echo "python ${script08} --runs ${run} --user ${user} --config-template ${configjson} --kerberos --hostname ${hostname} --headless"
    exit 0
fi

if [ -f ${script08} ]; then
    if [ "$doplot" == "true" ]; then
        echo "Running script with plotting enabled\n\n"
        python ${script08} --runs $run --user $user --config-template $configjson --kerberos --hostname ${hostname} --headless 
    else
        echo "Running script without plotting"
        python ${script08} --runs $run --user $user --config-template $configjson --kerberos --hostname ${hostname} 
    fi
else
    echo "Script not found: ${script08}"
fi

In [ ]:
from glob import glob
try: 
    wfset_full = load_structured_waveformset(f"{output_dir}/processed/run{run:0d}_{dettype}/processed_merged_run{run:06d}_structured_{dettype}.hdf5", max_waveforms=nwaveforms)
except:
    files = glob(f"{output_dir}/processed/run{run:06d}_{dettype}/processed_*_run{run:06d}_*_{dettype}.hdf5")
    print(files)
    files = files[0]
    wfset_full = load_structured_waveformset(files, max_waveforms=nwaveforms)


In [ ]:
filtering = 2
baseline = SBaseline(threshold= 25, baselinefinish=240, default_filtering=filtering, minimumfrac=0.67)

ip = IPDict(
    baseline_method="SBaseline",      # ← NEW (or "Mean", "Fit", …)
    int_ll=254, int_ul=270,
    amp_ll=254, amp_ul=270,
    baseliner=baseline,
    baseline_limits=[0,240],
    onlyoptimal=True
)
_ = wfset_full.analyse("std", BasicWfAna, ip,
                  analysis_kwargs={},
                  checks_kwargs=dict(points_no=wfset_full.points_per_wf),
                  overwrite=True,
                  show_progress=True
                 )


In [ ]:
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid
def remove_bad_baseline(waveform:Waveform) -> bool:
    # comment the two lines below to not remove any waveform
    #if waveform.analyses["std"].result['amplitude'] is np.nan:
    #    return False
    return True
    
wfset = WaveformSet.from_filtered_WaveformSet(wfset_full, remove_bad_baseline)


print(f"After removing: {len(wfset.waveforms)} waveforms")
test = ChannelWsGrid.clusterize_waveform_set(wfset)
print("By channel..")
for ep, v in test.items():
    print("Endpoint:", ep)
    for ch, wfs in v.items():
        print(f"Ch: {ch}, {len(wfs.waveforms)}")

In [ ]:
argsheat = dict(
    mode="heatmap",
    analysis_label="std",
    adc_range_above_baseline=200,
    adc_range_below_baseline=-50,
    adc_bins=100,
    time_bins=wfset.points_per_wf//2,
    filtering=2,
    share_y_scale=False,
    share_x_scale=True,
    wfs_per_axes=5000,
    zlog=True
)
detector=group1
for n, g in np02_gen_grids(wfset, detector).items():
    plot_grid(chgrid=g, title=n, html=None, detector=detector, **argsheat)

detector=group2
for n, g in np02_gen_grids(wfset, detector).items():
    plot_grid(chgrid=g, title=n, html=None, detector=detector, **argsheat)
    
detector=group3
for n, g in np02_gen_grids(wfset, detector).items():
    plot_grid(chgrid=g, title=n, html=None, detector=detector, **argsheat)
    
detector=group4
for n, g in np02_gen_grids(wfset, detector).items():
    plot_grid(chgrid=g, title=n, html=None, detector=detector, **argsheat)

In [ ]:
from waffles.plotting.plot import plot_CustomChannelGrid
from plotly import graph_objects as go
import plotly.subplots as psu
import numpy as np

def genhist(wfset:WaveformSet, figure:go.Figure, row, col):
    values = [wf.analyses["std"].result['integral'] for wf in wfset.waveforms]
    bins = np.linspace(-50e3, 50e3, 100)

    figure.add_trace(
        go.Histogram(
            x=values,
            xaxis='x',
            xbins=dict(start=bins[0], end=bins[-1], size=(bins[1] - bins[0])),
            autobinx=False,
            
        ),
        row=row, col=col
    )
def plothist(wfset:WaveformSet, detector:str):

    for n, g in np02_gen_grids(wfset, detector, rows=2).items():
        rows, cols= g.ch_map.rows, g.ch_map.columns
                                                        
        subtitles = g.titles                           
                                                        
        fig = psu.make_subplots(                            
            rows=rows,                                      
            cols=cols,                                      
            subplot_titles=subtitles,                       
        )                                                   

        plot_CustomChannelGrid(g, genhist, figure=fig)
    fig.update_layout(title=n, template="plotly_white",
                      width=1000, height=800, showlegend=True)
    fig.show()
    #chgrid=g, detmap=m, title=n, html=None, detector=detector, **argsheat)

detector=group1
plothist(wfset, detector)
detector=group2
plothist(wfset, detector)
detector=group3
plothist(wfset, detector)
detector=group4
plothist(wfset, detector)

In [ ]:
# Example of how to test things...
detector=[ "C3(2)"]
argsheat["adc_range_above_baseline"] = 20
argsheat["adc_range_below_baseline"] = -5
argsheat["adc_bins"] = 100
for n, (g, m) in np02_gen_grids(wfset, detector).items():
    plot_grid(chgrid=g, detmap=m, title=n, html=None, detector=detector, **argsheat)